In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping

/Users/brianconnor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [ ]:
# this notebook is a mess but here's what it includes

- a preliminary NN of all features thrown together, NaNs dropped, .2 MAE away from the top score on Driven Data
- a secondary NN of polynomial'ed weather features plus the other regular features, 1.7 MAE away from top
- a (mostly complete) function on creating a submission dataframe and csv from a few different inputs

What needs to be done:
- filling in NaN values in the test holdout because Driven Data will only accept submissions with all predictions
    - thinking about filling in values based on an average of the same week every year, or possibly with a value from one month before or after

In [2]:
train_csv = "./data/train.csv"

In [3]:
train = pd.read_csv(train_csv)

In [4]:
train.head()

,Unnamed: 0,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [5]:
train.drop("Unnamed: 0", axis=1, inplace=True)

In [6]:
# going to drop the week start date column because it is redundant
train.drop("week_start_date", axis=1, inplace=True)

In [7]:
pd.set_option('display.max_columns', 25)

In [8]:
train = pd.get_dummies(train)

In [9]:
train.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,city_iq,city_sj
0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,0,1
1,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,1
2,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4,0,1
3,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3,0,1
4,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6,0,1


#### The gameplan:
Using NN to create an "index" from the pixel index. Polynomial all the weather features. lasso

But first, just one off the cuff NN

#### Neural Network

In [10]:
target = "total_cases"
columns = [x for x in train.columns if x != target]

In [11]:
neuralX = train[columns]
neuralY = train[target]

In [12]:
neuralX_train, neuralX_test, neuraly_train, neuraly_test = train_test_split(neuralX, neuralY)

In [13]:
neural_ss = StandardScaler()
neuralXs_train = neural_ss.fit_transform(neuralX_train)
neuralXs_test = neural_ss.transform(neuralX_test)

In [14]:
neuralXs_train.shape

(899, 24)

In [15]:
network = Sequential()

network.add(Dense(24, activation="relu", input_dim=neuralXs_train.shape[1], kernel_regularizer=regularizers.l1(.01)))
network.add(Dropout(.5))
network.add(Dense(576, activation="relu", kernel_regularizer=regularizers.l1(.01)))
network.add(Dropout(.5))
network.add(Dense(24, activation="relu", kernel_regularizer=regularizers.l1(.01)))
network.add(Dropout(.5))
network.add(Dense(1, activation=None, kernel_regularizer=regularizers.l1(.01)))

network.compile(loss="mean_absolute_error", optimizer="adam")

network.fit(neuralXs_train, neuraly_train, validation_data=(neuralXs_test, neuraly_test), epochs=500)


Train on 899 samples, validate on 300 samples
Epoch 1/500
899/899 [==============================] - 1s 660us/step - loss: 32.8875 - val_loss: 26.9852
Epoch 2/500
899/899 [==============================] - 0s 63us/step - loss: 26.1717 - val_loss: 21.8031
Epoch 3/500
899/899 [==============================] - 0s 52us/step - loss: 22.4336 - val_loss: 19.3650
Epoch 4/500
899/899 [==============================] - 0s 56us/step - loss: 20.7284 - val_loss: 18.0079
Epoch 5/500
899/899 [==============================] - 0s 60us/step - loss: 19.9838 - val_loss: 17.2581
Epoch 6/500
899/899 [==============================] - 0s 56us/step - loss: 18.9190 - val_loss: 16.8135
Epoch 7/500
899/899 [==============================] - 0s 59us/step - loss: 18.4538 - val_loss: 16.4992
Epoch 8/500
899/899 [==============================] - 0s 57us/step - loss: 18.4126 - val_loss: 16.2708
Epoch 9/500
899/899 [==============================] - 0s 54us/step - loss: 18.1292 - val_loss: 16.1151
Epoch 10/500
899/

899/899 [==============================] - 0s 54us/step - loss: 15.5687 - val_loss: 13.6791
Epoch 156/500
899/899 [==============================] - 0s 57us/step - loss: 15.6304 - val_loss: 13.6479
Epoch 157/500
899/899 [==============================] - 0s 59us/step - loss: 15.2464 - val_loss: 13.6666
Epoch 158/500
899/899 [==============================] - 0s 52us/step - loss: 15.0902 - val_loss: 13.6804
Epoch 159/500
899/899 [==============================] - 0s 53us/step - loss: 15.5522 - val_loss: 13.6314
Epoch 160/500
899/899 [==============================] - 0s 57us/step - loss: 15.2953 - val_loss: 13.6291
Epoch 161/500
899/899 [==============================] - 0s 58us/step - loss: 15.4425 - val_loss: 13.6667
Epoch 162/500
899/899 [==============================] - 0s 56us/step - loss: 15.6364 - val_loss: 13.6595
Epoch 163/500
899/899 [==============================] - 0s 54us/step - loss: 15.3417 - val_loss: 13.6728
Epoch 164/500
899/899 [==============================] - 0s 

899/899 [==============================] - 0s 53us/step - loss: 14.9563 - val_loss: 13.3340
Epoch 310/500
899/899 [==============================] - 0s 55us/step - loss: 15.3000 - val_loss: 13.3422
Epoch 311/500
899/899 [==============================] - 0s 53us/step - loss: 15.0217 - val_loss: 13.3634
Epoch 312/500
899/899 [==============================] - 0s 51us/step - loss: 15.0032 - val_loss: 13.3486
Epoch 313/500
899/899 [==============================] - 0s 54us/step - loss: 14.7312 - val_loss: 13.3639
Epoch 314/500
899/899 [==============================] - 0s 52us/step - loss: 15.2421 - val_loss: 13.4216
Epoch 315/500
899/899 [==============================] - 0s 53us/step - loss: 15.3055 - val_loss: 13.4122
Epoch 316/500
899/899 [==============================] - 0s 53us/step - loss: 15.3357 - val_loss: 13.3743
Epoch 317/500
899/899 [==============================] - 0s 52us/step - loss: 15.3136 - val_loss: 13.3543
Epoch 318/500
899/899 [==============================] - 0s 

899/899 [==============================] - 0s 55us/step - loss: 14.8227 - val_loss: 13.2679
Epoch 464/500
899/899 [==============================] - 0s 54us/step - loss: 15.1550 - val_loss: 13.2861
Epoch 465/500
899/899 [==============================] - 0s 57us/step - loss: 14.7227 - val_loss: 13.2676
Epoch 466/500
899/899 [==============================] - 0s 56us/step - loss: 14.7982 - val_loss: 13.3039
Epoch 467/500
899/899 [==============================] - 0s 56us/step - loss: 14.8415 - val_loss: 13.2805
Epoch 468/500
899/899 [==============================] - 0s 57us/step - loss: 14.7728 - val_loss: 13.2868
Epoch 469/500
899/899 [==============================] - 0s 56us/step - loss: 15.0991 - val_loss: 13.2661
Epoch 470/500
899/899 [==============================] - 0s 65us/step - loss: 15.2764 - val_loss: 13.2793
Epoch 471/500
899/899 [==============================] - 0s 62us/step - loss: 14.6234 - val_loss: 13.2774
Epoch 472/500
899/899 [==============================] - 0s 

In [16]:
# neural networks are like cheating
# the high score on drivendata is 13 MAE

#### Engineering weather features

In [17]:
pd.set_option("display.max_columns",50)
train.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,city_iq,city_sj
0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4,0,1
1,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5,0,1
2,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4,0,1
3,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3,0,1
4,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6,0,1


In [18]:
non_weather = "year weekofyear ndvi_ne ndvi_nw ndvi_se ndvi_sw total_cases city_iq city_sj".split()
weather_features = [x for x in train.columns if x not in non_weather]

In [19]:
weather_features

['precipitation_amt_mm',
 'reanalysis_air_temp_k',
 'reanalysis_avg_temp_k',
 'reanalysis_dew_point_temp_k',
 'reanalysis_max_air_temp_k',
 'reanalysis_min_air_temp_k',
 'reanalysis_precip_amt_kg_per_m2',
 'reanalysis_relative_humidity_percent',
 'reanalysis_sat_precip_amt_mm',
 'reanalysis_specific_humidity_g_per_kg',
 'reanalysis_tdtr_k',
 'station_avg_temp_c',
 'station_diur_temp_rng_c',
 'station_max_temp_c',
 'station_min_temp_c',
 'station_precip_mm']

In [20]:
pf = PolynomialFeatures(degree=2, include_bias=False)
pf_features = pd.DataFrame(data=pf.fit_transform(train[weather_features]), columns=pf.get_feature_names(weather_features))


In [21]:
pf_features.shape

(1199, 152)

In [22]:
train_ex = train

In [23]:
train_ex.drop(labels=weather_features, axis=1, inplace=True)

In [24]:
train_ex = train_ex.join(pf_features, how="outer")

In [25]:
# setting up the data
engineered_feats = [x for x in train_ex.columns if x != target]
X2 = train_ex[engineered_feats]
y2 = train_ex[target]

In [26]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2)

In [27]:
ss2 = StandardScaler()
X2s_train = ss2.fit_transform(X2_train)
X2s_test = ss2.fit_transform(X2_test)

In [49]:
X2s_train

array([[-0.81979664,  0.70744439, -0.73018063, ...,  1.50168172,
        -0.47659598, -0.28315013],
       [-0.43980325,  0.70744439, -1.06858815, ...,  1.50168172,
        -0.04920945, -0.21420322],
       [-0.24980656,  0.50442487,  0.56962577, ...,  1.15369547,
        -0.52931594, -0.28749805],
       ...,
       [ 0.32018353, -0.37532643,  0.3796745 , ...,  0.08855495,
         2.12429773,  1.19128083],
       [-1.95977682, -0.91671184,  1.03486095, ...,  0.02443326,
        -0.28450256, -0.2498849 ],
       [ 0.89017362,  0.84279075, -1.17216028, ...,  1.9287754 ,
        -0.68580592, -0.29797305]])

In [29]:
y2_train.shape

(899,)

In [30]:
network2 = Sequential()

network2.add(Dense(976, activation="relu", input_dim=X2s_train.shape[1], kernel_regularizer=regularizers.l1(.01)))
network2.add(Dropout(.5))
network2.add(Dense(976, activation="relu", kernel_regularizer=regularizers.l1(.01)))
network2.add(Dropout(.5))
network2.add(Dense(31, activation="relu", kernel_regularizer=regularizers.l1(.01)))
network2.add(Dropout(.5))
network2.add(Dense(1, activation=None, kernel_regularizer=regularizers.l1(.01)))

network2.compile(loss="mean_absolute_error", optimizer="adam")

network2.fit(X2s_train, y2_train, validation_data=(X2s_test, y2_test), epochs=500)

Train on 899 samples, validate on 300 samples
Epoch 1/500
899/899 [==============================] - 1s 1ms/step - loss: 241.0017 - val_loss: 136.0312
Epoch 2/500
899/899 [==============================] - 1s 621us/step - loss: 85.8489 - val_loss: 45.8930
Epoch 3/500
899/899 [==============================] - 1s 664us/step - loss: 36.0821 - val_loss: 25.8683
Epoch 4/500
899/899 [==============================] - 0s 511us/step - loss: 24.7445 - val_loss: 20.6269
Epoch 5/500
899/899 [==============================] - 0s 552us/step - loss: 21.9003 - val_loss: 19.0221
Epoch 6/500
899/899 [==============================] - 0s 471us/step - loss: 20.6417 - val_loss: 18.1815
Epoch 7/500
899/899 [==============================] - 0s 488us/step - loss: 19.9815 - val_loss: 17.5656
Epoch 8/500
899/899 [==============================] - 0s 481us/step - loss: 19.2433 - val_loss: 17.2531
Epoch 9/500
899/899 [==============================] - 0s 491us/step - loss: 19.1070 - val_loss: 16.9384
Epoch 10/

Epoch 78/500
899/899 [==============================] - 0s 519us/step - loss: 17.0826 - val_loss: 15.3382
Epoch 79/500
899/899 [==============================] - 0s 478us/step - loss: 17.3853 - val_loss: 15.2109
Epoch 80/500
899/899 [==============================] - 0s 463us/step - loss: 17.1406 - val_loss: 15.3411
Epoch 81/500
899/899 [==============================] - 0s 463us/step - loss: 17.2063 - val_loss: 15.1793
Epoch 82/500
899/899 [==============================] - 0s 460us/step - loss: 17.0576 - val_loss: 15.1269
Epoch 83/500
899/899 [==============================] - 0s 463us/step - loss: 17.1276 - val_loss: 15.2351
Epoch 84/500
899/899 [==============================] - 0s 468us/step - loss: 17.2833 - val_loss: 15.2520
Epoch 85/500
899/899 [==============================] - 0s 478us/step - loss: 16.9151 - val_loss: 15.2728
Epoch 86/500
899/899 [==============================] - 0s 470us/step - loss: 17.4604 - val_loss: 15.2447
Epoch 87/500
899/899 [========================

Epoch 155/500
899/899 [==============================] - 0s 497us/step - loss: 16.8979 - val_loss: 14.8987
Epoch 156/500
899/899 [==============================] - 0s 462us/step - loss: 16.8551 - val_loss: 15.0058
Epoch 157/500
899/899 [==============================] - 0s 452us/step - loss: 16.6568 - val_loss: 14.9832
Epoch 158/500
899/899 [==============================] - 0s 455us/step - loss: 16.8685 - val_loss: 14.9273
Epoch 159/500
899/899 [==============================] - 0s 463us/step - loss: 16.8184 - val_loss: 15.0716
Epoch 160/500
899/899 [==============================] - 0s 463us/step - loss: 17.0072 - val_loss: 15.0069
Epoch 161/500
899/899 [==============================] - 0s 467us/step - loss: 16.6374 - val_loss: 14.9428
Epoch 162/500
899/899 [==============================] - 0s 464us/step - loss: 16.7606 - val_loss: 14.9852
Epoch 163/500
899/899 [==============================] - 0s 459us/step - loss: 16.7193 - val_loss: 14.9466
Epoch 164/500
899/899 [==============

899/899 [==============================] - 0s 543us/step - loss: 16.5783 - val_loss: 14.6968
Epoch 308/500
899/899 [==============================] - 0s 507us/step - loss: 16.4402 - val_loss: 15.0000
Epoch 309/500
899/899 [==============================] - 0s 491us/step - loss: 16.4347 - val_loss: 14.6986
Epoch 310/500
899/899 [==============================] - 0s 482us/step - loss: 16.7020 - val_loss: 14.8194
Epoch 311/500
899/899 [==============================] - 0s 484us/step - loss: 16.5542 - val_loss: 14.8154
Epoch 312/500
899/899 [==============================] - 0s 484us/step - loss: 16.7822 - val_loss: 14.6630
Epoch 313/500
899/899 [==============================] - 0s 515us/step - loss: 16.8396 - val_loss: 14.6844
Epoch 314/500
899/899 [==============================] - 0s 476us/step - loss: 16.8640 - val_loss: 14.9096
Epoch 315/500
899/899 [==============================] - 0s 488us/step - loss: 16.1879 - val_loss: 14.7728
Epoch 316/500
899/899 [============================

899/899 [==============================] - 0s 486us/step - loss: 16.3883 - val_loss: 14.5373
Epoch 460/500
899/899 [==============================] - 0s 466us/step - loss: 16.4871 - val_loss: 14.4469
Epoch 461/500
899/899 [==============================] - 0s 479us/step - loss: 16.5779 - val_loss: 14.5610
Epoch 462/500
899/899 [==============================] - 0s 467us/step - loss: 16.5000 - val_loss: 14.6015
Epoch 463/500
899/899 [==============================] - 0s 472us/step - loss: 16.1020 - val_loss: 14.7980
Epoch 464/500
899/899 [==============================] - 0s 464us/step - loss: 16.3474 - val_loss: 14.7338
Epoch 465/500
899/899 [==============================] - 0s 468us/step - loss: 16.6673 - val_loss: 14.6236
Epoch 466/500
899/899 [==============================] - 0s 466us/step - loss: 16.5602 - val_loss: 14.9717
Epoch 467/500
899/899 [==============================] - 0s 468us/step - loss: 16.5172 - val_loss: 14.5289
Epoch 468/500
899/899 [============================

### creating submissions

In [98]:
test_csv = "./data/dengue_features_test.csv"
test = pd.read_csv(test_csv)

In [72]:
test.drop("week_start_date", axis=1, inplace=True)
test.dropna(inplace=True)
test = pd.get_dummies(test)

In [73]:
pf_test = pf.transform(test[weather_features])

In [74]:
pf_test

array([[7.86000000e+01, 2.98492857e+02, 2.98550000e+02, ...,
        4.70890000e+02, 1.63184000e+03, 5.65504000e+03],
       [1.25600000e+01, 2.98475714e+02, 2.98557143e+02, ...,
        4.92840000e+02, 7.61460000e+02, 1.17649000e+03],
       [7.60000000e-01, 2.99780000e+02, 2.99671429e+02, ...,
        5.42890000e+02, 1.95953000e+03, 7.07281000e+03],
       ...,
       [7.89600000e+01, 2.95831429e+02, 2.96607143e+02, ...,
        4.66560000e+02, 2.01312000e+03, 8.68624000e+03],
       [3.95400000e+01, 2.95778571e+02, 2.97400000e+02, ...,
        4.75240000e+02, 7.43380000e+02, 1.16281000e+03],
       [5.18000000e+01, 2.97372857e+02, 2.99000000e+02, ...,
        4.84000000e+02, 3.27800000e+02, 2.22010000e+02]])

In [75]:
test_weather = pd.DataFrame(data=pf_test, columns=pf.get_feature_names(weather_features))

In [76]:
test.head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,city_iq,city_sj
0,2008,18,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,294.527143,301.1,296.4,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2,0,1
1,2008,19,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,294.395714,300.8,296.7,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3,0,1
4,2008,22,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,294.760000,302.3,297.3,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1,0,1
5,2008,23,-0.0440,-0.030467,0.132000,0.083529,71.17,299.768571,299.728571,295.314286,301.9,297.6,22.55,76.557143,71.17,16.667143,2.857143,28.000000,5.171429,32.8,25.0,27.7,0,1
6,2008,24,-0.0443,-0.024925,0.132271,0.159157,48.99,300.062857,300.007143,295.650000,302.4,297.5,13.10,76.844286,48.99,17.010000,3.157143,27.400000,6.042857,31.1,23.3,91.7,0,1


In [77]:
test.drop(labels=weather_features, axis=1, inplace=True)

In [78]:
test.shape

(353, 8)

In [79]:
test_weather.shape

(353, 152)

In [80]:
test.reset_index(drop=True, inplace=True)
test_weather.reset_index(drop=True, inplace=True)

test['id'] = test.index
test_weather['id'] = test_weather.index

In [81]:
test_weather.shape

(353, 153)

In [82]:
test.shape

(353, 9)

In [83]:
test_ex = test.merge(test_weather, how='left', on='id')

In [84]:
test_ex.drop("id", axis=1, inplace=True)

In [107]:
test_ex.shape

(353, 160)

In [86]:
test_ex = ss2.transform(test_ex)

In [106]:
test_ex

array([[ 1.20659642, -0.54858183, -1.18023178, ..., -0.23140573,
         0.8027309 ,  0.26303983],
       [ 1.20659642, -0.48288341, -1.17392203, ...,  0.11581201,
        -0.10552695, -0.26871775],
       [ 1.20659642, -0.28578814, -0.64951201, ...,  0.90753174,
         1.14468156,  0.43137779],
       ...,
       [ 2.10302765, -0.15439129,  0.62595831, ..., -0.29990017,
         1.20060373,  0.6229473 ],
       [ 2.10302765, -0.08869287,  1.12863475, ..., -0.16259492,
        -0.12439377, -0.27034204],
       [ 2.10302765, -0.02299445,  1.33221591, ..., -0.0240242 ,
        -0.55805929, -0.38204729]])

In [87]:
network2_preds = network2.predict(test_ex)

In [88]:
len(network2_preds)

353

In [89]:
network2_df = pd.DataFrame(test["city"])

KeyError: 'city'

In [90]:
network2_df["year"] = pd.DataFrame(test["year"])

In [91]:
network2_df["weekofyear"] = pd.DataFrame(test["weekofyear"])

In [92]:
network2_df["total_cases"] = network2_preds

ValueError: Length of values does not match length of index

In [94]:
# im going to pipeline this because it's making me want to explode

In [203]:
test_csv = "./data/dengue_features_test.csv"
test = pd.read_csv(test_csv)

In [220]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 24 columns):
city                                     416 non-null object
year                                     416 non-null int64
weekofyear                               416 non-null int64
week_start_date                          416 non-null object
ndvi_ne                                  373 non-null float64
ndvi_nw                                  405 non-null float64
ndvi_se                                  415 non-null float64
ndvi_sw                                  415 non-null float64
precipitation_amt_mm                     414 non-null float64
reanalysis_air_temp_k                    414 non-null float64
reanalysis_avg_temp_k                    414 non-null float64
reanalysis_dew_point_temp_k              414 non-null float64
reanalysis_max_air_temp_k                414 non-null float64
reanalysis_min_air_temp_k                414 non-null float64
reanalysis_precip_amt_kg_per_m2  

In [214]:
def pred_maker(test_set, model, csv_name, polyfunc=None, scalerfunc=None):
    # preliminary preparation of test set
    test_set.dropna(inplace=True)
    test_set.drop("week_start_date", axis=1, inplace=True)
    
    # this is to pull from for later in the prediction dataframe
    city_df = test_set["city"]
    year_df = test_set["year"]
    weekofyear_df = test_set["weekofyear"]
    
    #dummying cities
    test_set = pd.get_dummies(test_set)
    
    # prepping weather features
    non_weather = "year weekofyear ndvi_ne ndvi_nw ndvi_se ndvi_sw total_cases city_iq city_sj".split()
    weather_features = [x for x in test_set.columns if x not in non_weather]
    
    # if there is a polynomial features function
    if polyfunc is not None:
        # calling the func and making a dataframe
        pf_test = polyfunc.transform(test_set[weather_features])
        test_weather = pd.DataFrame(data=pf_test, columns=pf.get_feature_names(weather_features))
        
        test_set.drop(labels=weather_features, axis=1, inplace=True)
        
        # fixing the indices
        test_set.reset_index(drop=True, inplace=True)
        test_weather.reset_index(drop=True, inplace=True)

        test_set['id'] = test_set.index
        test_weather['id'] = test_weather.index
        
        #merging dataframes
        test_set = test_set.merge(test_weather, how='left', on='id')
        test_set.drop("id", axis=1, inplace=True)
        
    # if there is a scaler function
    if scalerfunc is not None:
        test_set = scalerfunc.transform(test_set)
    
    # making predictions
    pred_array = model.predict(test_set)
    
    #setting up prediction dataframe
    pred_df = pd.DataFrame()
    pred_df["city"] = city_df
    pred_df["year"] = year_df
    pred_df["weekofyear"] = weekofyear_df
    pred_df["total_cases"] = pred_array
    
    pred_df.to_csv("./submissions/"+csv_name, index=False)
    
    return pred_df

In [190]:
func_test = pred_maker(test, network2, "weather_poly_nn", pf, ss2)

In [191]:
func_test

,city,year,weekofyear,total_cases
0,sj,2008,18,4.894512
1,sj,2008,19,5.421768
4,sj,2008,22,5.312186
5,sj,2008,23,8.374026
6,sj,2008,24,8.558281
8,sj,2008,26,12.078225
9,sj,2008,27,16.812138
10,sj,2008,28,12.495796
12,sj,2008,30,18.654673
13,sj,2008,31,19.560387
